論文<br>
http://arxiv.org/abs/1912.04958<br>
<br>
GitHub<br>
https://github.com/NVlabs/stylegan2-ada-pytorch

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# 環境セットアップ

In [ ]:
!nvidia-smi

## GithubからCode clone

In [ ]:
%cd /content

!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
# for align face
!git clone https://github.com/adamian98/pulse.git

## ライブラリのインストール

In [ ]:
!pip install moviepy

# ninja
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip > /dev/null
!sudo unzip ninja-linux.zip -d /usr/local/bin/ > /dev/null
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force > /dev/null

## ライブラリのインポート

In [ ]:
%cd /content/stylegan2-ada-pytorch

import cv2
import numpy as np
from PIL import Image
import dlib
from matplotlib import pyplot as plt
import torch
import dnnlib
import legacy
import imageio
from tqdm.notebook import tqdm

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

# テスト画像のセットアップ

In [ ]:
%cd /content/stylegan2-ada-pytorch
!rm -rf inputs
!mkdir -p inputs/img inputs/align

!wget -c https://hips.hearstapps.com/amapn-prod-jp.s3.amazonaws.com/elljp/elle-mig/images/culture/feature/daniel_radcliffe16_1129/3/1754607-1-jpn-JP/2002-10-25.jpg \
      -O ./inputs/img/src.jpg

!wget -c https://www.tvgroove.com/wp-content/uploads/2020/03/Daniel-Radcliffe.jpg \
      -O ./inputs/img/dst.jpg

# Preprocess

In [ ]:
NETWORK = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"
SOURCE_NAME = '/content/stylegan2-ada-pytorch/src.jpg'
TARGET_NAME = '/content/stylegan2-ada-pytorch/dst.jpg'

In [ ]:
%cd /content/pulse
!python align_face.py \
  -input_dir /content/stylegan2-ada-pytorch/inputs/img \
  -output_dir /content/stylegan2-ada-pytorch/inputs/align \
  -output_size 1024 \
  -seed 12 \

In [ ]:
%cd /content/stylegan2-ada-pytorch

align_src = cv2.imread('./inputs/align/src_0.png')
img = cv2.cvtColor(align_src, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

align_dst = cv2.imread('./inputs/align/dst_0.png')
img = cv2.cvtColor(align_dst, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

# Invert to GAN

In [ ]:
%cd /content/stylegan2-ada-pytorch

!python projector.py \
  --save-video 0 \
  --num-steps 1000 \
  --outdir=outputs/src \
  --target=inputs/align/src_0.png \
  --network={NETWORK}

In [ ]:
%cd /content/stylegan2-ada-pytorch

!python projector.py \
  --save-video 0 \
  --num-steps 1000 \
  --outdir=outputs/dst \
  --target=inputs/align/dst_0.png \
  --network={NETWORK}

In [ ]:
%cd /content/stylegan2-ada-pytorch

img_gan_source = cv2.imread('./outputs/src/proj.png')
img = cv2.cvtColor(img_gan_source, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title('source-gan')
plt.show()

img_gan_target = cv2.imread('./outputs/dst/proj.png')
img = cv2.cvtColor(img_gan_target, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title('target-gan')
plt.show()

# Morphing

In [ ]:
STEPS = 100
FPS = 20
FREEZE_STEPS = 20

In [ ]:
%cd /content/stylegan2-ada-pytorch

lvec1 = np.load('./outputs/src/projected_w.npz')['w']
lvec2 = np.load('./outputs/dst/projected_w.npz')['w']

network_pkl = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"
device = torch.device('cuda')
with dnnlib.util.open_url(network_pkl) as fp:
    G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device) # type: ignore

diff = lvec2 - lvec1
step = diff / STEPS
current = lvec1.copy()
target_uint8 = np.array([1024,1024,3], dtype=np.uint8)

video = imageio.get_writer('./movie.mp4', mode='I', fps=FPS, codec='libx264', bitrate='16M')

for j in tqdm(range(STEPS)):
  z = torch.from_numpy(current).to(device)
  synth_image = G.synthesis(z, noise_mode='const')
  synth_image = (synth_image + 1) * (255/2)
  synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()

  repeat = FREEZE_STEPS if j==0 or j==(STEPS-1) else 1
   
  for i in range(repeat):
    video.append_data(synth_image)
  current = current + step


video.close()

In [ ]:
clip = VideoFileClip('./movie.mp4')
clip = resize(clip, height=420)
clip.ipython_display()